# Initial set up


In [1]:
#Using Google colab secrets to store keys.
from google.colab import userdata

import urllib.parse

import requests

import json

lingq_api_key=userdata.get('LINGQ_API_KEY');
headers = {
    'Authorization': f'Token {lingq_api_key}',
    'Content-Type': 'application/json'
}


# Helper Functions

In [2]:
def get_json_response (url):
  response = requests.get(url, headers=headers)
  return response.json()

def print_json (json_parsed):
  print (json.dumps(json_parsed, indent = 4))



# Languages


In [3]:
languages = get_json_response('https://www.lingq.com/api/v2/languages/')
languages_with_known_words = [(f'{item["title"]} ({item["knownWords"]})', item["code"]) for item in languages if item["knownWords"] != 0]
print_json(languages_with_known_words)



[
    [
        "Polish (1132)",
        "pl"
    ],
    [
        "English (169)",
        "en"
    ]
]


# Select Language

In [4]:
import ipywidgets as widgets
from IPython.display import display
lang_selector = widgets.Dropdown(
    options=languages_with_known_words,
    value=languages_with_known_words[0][1],
    description='Language:',
)
display(lang_selector)

Dropdown(description='Language:', options=(('Polish (1132)', 'pl'), ('English (169)', 'en')), value='pl')

In [5]:
language_code = lang_selector.value
language_code

'pl'

# Courses for Language

In [6]:

courses_in_language = get_json_response (f'https://www.lingq.com/api/v2/{language_code}/collections/my/')

course_selector = widgets.Dropdown(
    options= [(item["title"], item["id"]) for item in courses_in_language["results"]],
    value = courses_in_language["results"][0]["id"],
    description = "Course:")
display(course_selector)



Dropdown(description='Course:', options=(('A Polish Family', 1424980), ('Daily Polish Listening', 1456660), ('…

In [11]:
course_id = course_selector.value
course_id


1440209

# Lessons from Course

In [12]:
lessons_in_course = get_json_response(f'https://www.lingq.com/api/v2/{language_code}/collections/{course_id}')
lesson_selector = widgets.Dropdown(
    options= [(item["title"], item["url"]) for item in lessons_in_course["lessons"]],
    value = lessons_in_course["lessons"][0]["url"],
    description = "Lesson:")
display(lesson_selector)


Dropdown(description='Lesson:', options=(('001A-Marta próbowała usnąć, ale nie mogła.', 'https://www.lingq.com…

In [13]:
lesson_url = lesson_selector.value

# Lesson

In [16]:

lesson = get_json_response(lesson_url)
#print_json(lesson["tokenizedText"])
sentences = [sentence[0]["text"] for sentence in lesson["tokenizedText"] if not ("opentag" in sentence[0]["tokens"][0])]
text = " ".join(sentences)
text

'Cześć! Witam i zapraszam na pierwszą historyjkę pod tytułem Marta nie może spać. Posłuchaj uważnie historyjki kilka razy, a potem zapraszam cię na pytania i odpowiedzi. Zaczynajmy! Marta poszła spać. Była jedenasta wieczorem. Ona wyłączyła światło i położyła się do łóżka. W pokoju było ciemno i cicho. Marta próbowała usnąć, ale nie mogła. Odwróciła się i wyłączyła światło. Wzięła swoją książkę. Marta otworzyła książkę i zaczęła ją czytać. To była dobra książka. Marta przeczytała jedną stronę. Potem przeczytała jeszcze jedną stronę. Po chwili poczuła się zmęczona. Marta zamknęła książkę i wyłączyła światło. Zamknęła oczy. Po chwili Marta już spała. Świetnie! To była nasza historyjka. Posłuchaj tej historyjki kilka razy, a później zapraszam cię na pytania i odpowiedzi.'

# Load spaCy (select Model appropriate for your language)


In [19]:
# Install spaCy and the small Polish language model
!pip install spacy

#Using Polish model - select a language model for your language from here.
#A small one will be sufficient.
# https://spacy.io/usage/models#languages
!python -m spacy download pl_core_news_sm


import spacy
from spacy import displacy

nlp = spacy.load("pl_core_news_sm")

2023-12-19 20:03:08.879314: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 20:03:08.879404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 20:03:08.881271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 20:03:10.348073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 25.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


# Natural Language Processing Using Spacy


In [25]:
doc = nlp(text)


In [27]:

import pandas as pd

cols = ("text", "lemma", "POS", "explain", "morphology")
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.morph]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)

df

,text,lemma,POS,explain,morphology
0,Cześć,cześć,INTJ,interjection,()
1,!,!,PUNCT,punctuation,(PunctType=Excl)
2,Witam,witać,VERB,verb,"(Aspect=Imp, Mood=Ind, Number=Sing, Person=1, ..."
3,i,i,CCONJ,coordinating conjunction,()
4,zapraszam,zapraszać,VERB,verb,"(Aspect=Imp, Mood=Ind, Number=Sing, Person=1, ..."
...,...,...,...,...,...
137,na,na,ADP,adposition,(AdpType=Prep)
138,pytania,pytanie,NOUN,noun,"(Case=Acc, Gender=Neut, Number=Plur)"
139,i,i,CCONJ,coordinating conjunction,()
140,odpowiedzi,odpowiedź,NOUN,noun,"(Case=Gen, Gender=Fem, Number=Plur)"


In [29]:
sentence_spans = list(doc.sents)
displacy.render(sentence_spans, style="dep", jupyter=True)